The following code is written in Sagemath version 9.8
and uses the libaries fpylll, numpy, and gmpy2.

sage:sage0_version()

'SageMath version 9.8, Release Date: 2023-02-11'<br><br>

License : GPL v2<br>
Authors : K.A.Draziotis (drazioti@gmail.com)<br>
          M. Adamoudis<br>
          D. Poulakis<br>

**Implementation of the attack**
<br>
**1.** First we generate $(n+1)$  ephemeral keys with $\delta_M-$ MSB and $\delta_L-$ LSB shared. Overall $\delta=\delta_M+\delta_L$ shared bits. We do not fix the shared MSB and LSB, we just generate them randomly. $\ell$ is usually 160 or 256, which is the length of $q$.

In [1]:
reset()

def bits(N):
    return floor(log(N,2))+1

def gen_q(ell):
    return random_prime(2^ell-1,False,2^(ell-1) )
    
def gen_nonces(ell,delta,dL,n):
    '''
    Input:
    ell : the binary length of q, Usually 160 or 256
    delta : the number of shared bits
    dL : the number of shared LSB 
    n : n+1 is the number of signatures
    
    Output: A quadraple : two lists with n+1 elements and two integers q and the secret key a
    The first is the binary form of the ephemeral keys and 
    the second is the ephemeral keys represented as integers.
    '''
    Bits=[]
    K=[]
    q = gen_q(ell)
    a = randint(1, q)
    dM = delta - dL
    MSB = random_vector(dM,2).list()
    LSB = random_vector(dL,2).list()
    #print(MSB)
    #print(LSB)
    for _ in range(n+1):
            o = random_vector(ell-delta,2).list()
            Bits.append(MSB+o+LSB)
    for j in range(n+1):
        ki=sum(2^(ell-i-1) * Bits[j][i] for i in range(ell))
        K.append(ki)
    #sanity check
    for j in range(n+1):
        if K[j]>q:
            return 0
    return Bits,K,q,a
    

**2.** We construct the $z_j=k_j-k_r,z_j'=\frac{z_j}{2^{\delta_L}}$ for $j\in\{1,2,...,n\}-\{r\}$

In [2]:
def gen_zjs(n,dL,K):
    import copy
    K_tmp=copy.deepcopy(K)
    minK=min(K_tmp)
    min_index=K_tmp.index(min(K_tmp))
    K_tmp.pop(min_index) # we remove from the list K_tmp the minimum element
    Z=[]          # we store z[j]
    Z_prime = []  # we store z'[j]
    Z = [K_tmp[j]-minK for j in range(n)]
    Z_prime = [x/2^dL for x in Z]
    return Z,Z_prime,min_index

**3.** We continue with the $A_j,B_j.$ These quantities satisfy the signing equation
$k_j+A_ja+B_j\equiv 0 \pmod{q}, j=0,1,...,n$

In [3]:
def gen_Aj_and_Bj(n,q,a,K):
    # a is the secret key
    # K is the list of n+1 ephemeral keys
    A=[]
    B=[]
    A = [randint(1, q) for i in range(n+1)]
    B = [int(mod(-K[i] - A[i]*a,q)) for i in range(n+1)]
    return A,B
    

**4.** Set min_index=r. We generate $C_j = (A_j-A_r)2^{-\delta_L}\pmod{q},\ D_j=(B_j-B_0) 2^{-\delta_L} \pmod{q}, j\in \{1,2,...,n\}-\{r\}$<br>
$C_j$ and $D_j$ must satisfy the congeunces $z_j'+C_ja+D_j\equiv 0 \pmod{q}.$


In [4]:
def gen_Cj_and_Dj(n,dL,Aj,Bj,r,q):
    # r is the min_index from K. That is K[r]=min{K[0],...,K[n]}.
    C=[]
    D=[]
    for i in [x for x in range(n+1) if x != r]:
        C.append(int( mod( (Aj[i]-Aj[r]) * 2^(-dL), q) ) )
        D.append(int( mod( (Bj[i]-Bj[r]) * 2^(-dL), q) ) )
    return C,D

In [5]:
def sanity_check(Zj_prime,K,Aj,Bj,Dj,Cj,q,a):
    # we make some sanity checks:
    # initially we check if Z_prime contains integers
    # then we check the validity of the equiavalences, 
    # [1]
    for x in Zj_prime:
        if x not in ZZ:
            print("Z_prime contains non integers")
            return
    print("Z_prime sanity check : True")

    # [2] sanity check for Aj,Bj
    for i in range(n+1):
        if mod(K[i]+Aj[i]*a+Bj[i],q)!=0:
            print("Aj,Bj do not satisfy the signing equation")
            return
    print("Aj,Bj, sanity check  : True")

    # [3] sanity check fot Cj,Dj
    for i in range(n):
        if mod(Zj_prime[i]+Cj[i]*a+Dj[i],q)!=0:
            print("Cj,Dj do not satisfy the signing equation")
            return
    print("Cj,Dj, sanity check  : True")
        
    

**5.** Construct the DSA matrix

In [6]:
def dsa_matrix(n,delta,q,Cj):
    Zero_Matrix=matrix(n,1)  # part of the last column
    C=2^(delta+1)*matrix(Cj) # part of the last row
    I=q*identity_matrix(n)
    B = 2^(delta+1)*I         # the upper left nxn part
    B_1=block_matrix([[B,Zero_Matrix]]) # the upper nxn part
    B_2=block_matrix([[C,1]]) # the last row
    B_3=block_matrix([[B_1],[B_2]]) # the desired matrix
    return B_3

**6.** We need Kannan algorithm for closest vector

In [7]:
'''
A = matrix([[9,0,0],[0,9,0],[2,3,2]])
A = A.LLL() # LLL reduction before we start the enumertation
t = [7,6,0]
n = A.dimensions()[0] # the number of rows
b = 5^2    # the desired distance of the form R^2
termination = 2^25 # the wors case for iterations in the algorithm
S = kannan_cvp(A,t,b,termination) # Enumeration for CVP based on Kannan's algorithm
[[S[i],LA.norm(S[i]-np.array(t))] for i in range(len(S))]

output:
[[[array([4, 6, 4])], 5.0],
[[array([ 7,  6, -2])], 2.0],
[[array([9, 9, 0])], 3.605551275463989]]
'''

from fpylll import *
from fpylll.fplll.gso import MatGSO
from fpylll.fplll.lll import LLLReduction
from fpylll import IntegerMatrix
import numpy as np
from numpy import linalg as LA



# if for some reason the target is not in the span, sagemath will raise an arithmetic error
def find_linear_combination(basis_matrix, target):
    basis = basis_matrix.rows() # the basis vectors
    dim = len(basis)            # basis is a list of sage vectors
    if len(target)!=dim:
        return "target vector does not have the right dimension"
    basis_sage = []
    basis_sage.append([vector(x) for x in basis])
    basis_sage = basis_sage[0]  
    K = QQ  # K is the rational field
    VS = (K ** (dim)).span_of_basis(basis_sage)  # we define the subspace generated by v[0],v[1],..,v[n-2]

    try:
        z = VS.coordinate_vector(vector(target))
        z = vector(list(z))
        return z
    except ArithmeticError:
        print("Oops! vector is not in the span.")
        return
    
def mat2fp(A):
    L = IntegerMatrix(A.shape[0],A.shape[1])
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            L[i,j] = int(A[i,j])
    return L

def sage2fp(A):
    L = IntegerMatrix(A.dimensions()[0],A.dimensions()[1])
    for i in range(A.dimensions()[0]):
        for j in range(A.dimensions()[1]):
            L[i,j] = int(A[i,j])
    return L

def kannan_cvp(B,t,b,termination): 
    ''' 
    B is a fpylll type matrix,
    t is the target vector,
    b is the square of the bound,
    after "termination" iterations the algorithm stops.
    The output prints only the indepedent points. 
    '''

    # step 1 of enumeration algorithm : compute ||b_i*||^2,mu_{j,i} for j>i
    I = []
    
    gm_basis,_ = B.gram_schmidt()
    t = find_linear_combination(gm_basis,t)
    
    B = mat2fp(np.matrix(B))
    M =  MatGSO(B) # this is floating point Gram-Schmidt
    M.update_gso()
    rows = B.nrows
    gram = [M.get_r(k,k) for k in range(rows)]  # ||b_k*||^2, k=0,...,rows-1
    
    mu = np.matrix([M.get_mu(k,j) for j in range(rows) for k in range(rows)]).reshape(rows,rows).transpose() #\mu_{i,j} for i>j
    # step 2: initilaization
    S = []
    l=np.zeros(rows+1)
    x=np.zeros(rows).astype(int) 
    c=np.zeros(rows)
    y=np.zeros(rows)
    j = -1
    # compute the coefficient of t with respect to GS basis
    x[rows-1] =  ceil(t[rows-1] - np.sqrt(b)/np.sqrt(gram[rows-1]))# change :this is a new line       
    # step 3: termination condition in case the algorithm is too slow
    i = rows-1; # change 1: i=0-->i=rows-1
    counter = 0
    while i<=rows-1:
        # a termination condition
        termination_condition = termination
        if counter==termination_condition:
            print("Termination condition activated. Maybe you need to increase it. Current value:",counter)
            break

        counter = counter + 1
        
        #line 4:
        c[i] = float( -sum(x[j]*mu[j,i] for j in range(i+1, rows)))
        l[i] = gram[i]*(x[i] - c[i] - t[i])**2 # change 2: add -t[i]
        sumli = sum(float(l[j]) for j in range(i,rows))    # square norm       
        #line 5
        if sumli<=b:
            if i==0:             
                j = j + 1
                S.append([sum(x[j]*np.array(B[j]) for j in range(rows))]) 
                x[0] = x[0] + 1                       
            #line 6
            else:                
                #line 7
                i = i - 1
                li_prime = sum(float(l[j]) for j in range(i+1,rows))
                x[i] = ceil(t[i] - sum(x[j]*mu[j,i] for j in range(i+1, rows)) - float(sqrt( (b-li_prime)/gram[i]) ) ) # change 3 : add t[i]             
        #line 8:
        else:
            if i==rows-1:
                break            
            if i<rows - 1:   
                i = i + 1
                x[i] = x[i] + 1 
    return S


**7.** We are now ready to provide the attack

In [8]:
# we generate : q,nonces which are <q and zj,zj's,Cj,Dj for j = 1,2,...,n 
# and Aj,Bj for j=0,1,....,n
import time
def attack(ell,delta,dL,n):
    T1=time.time()
    O=gen_nonces(ell,delta,dL,n)
    while O==0:
        O=gen_nonces(ell,delta,dL,n)
    _,K,q,a=O
    Zj,Zj_prime,min_index=gen_zjs(n,dL,K)
    Aj,Bj=gen_Aj_and_Bj(n,q,a,K)
    Cj,Dj=gen_Cj_and_Dj(n,dL,Aj,Bj,min_index,q)
    #sanity_check(Zj_prime,K,Aj,Bj,Dj,Cj,q,a)
    M=dsa_matrix(n,delta,q,Cj) # the dsa matrix
    ###############
    M = M.BKZ(block_size=8)                                            # LLL reduction before we start the enumeration
    t = [2^(delta+1)*Dj[i] + 2^ell for i in range(n)]+[0]  # target vector
    R2 = (2^ell*sqrt(n+1))**2                                # the desired distance of the form R^2
    termination = 2^25                                     # the worst case for iterations in the algorithm
    S = kannan_cvp(M,t,R2,termination)                     # Enumeration for CVP based on Kannan's algorithm
    Solutions = [S[i][0].tolist() for i in range(len(S))]
    a_negative = -a%q   
    for x in Solutions:
        if x[n]==a_negative: 
            print("private key is found")
            T2 = time.time()-T1
            print(time.time()-T1)
            return T2
    return 

In [ ]:
ell,dL=160,2
for delta,signatures in [(10,18)]:
    n = signatures - 1
    T=[]
    for i in range(10):
        print(i+1,"\n")
        Ti=attack(ell,delta,dL,n)
        T.append(Ti)
    print("Results")
    print("ell,delta,dL,signatures:",ell,delta,dL,signatures,":",np.mean(T),"\n")

1 

private key is found
1.287515640258789
2 

private key is found
0.6137542724609375
3 

private key is found
0.5229959487915039
4 

private key is found
115.21785426139832
5 

private key is found
0.3987858295440674
6 

private key is found
3.083454132080078
7 

private key is found
1.830714225769043
8 

private key is found
19.774173259735107
9 

private key is found
0.5341732501983643
10 

